<a href="https://colab.research.google.com/github/asking28/offenseval2020/blob/master/offens_2020_danish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!pip install focal-loss
!pip install keras-tcn==2.8.3
!pip install keras-multi-head


  Created wheel for keras-multi-head: filename=keras_multi_head-0.22.0-cp36-none-any.whl size=15371 sha256=bcf41dd16a4d1e6443b14ef4c5ebcc03a0452ba04ab3ed4e5813344ec64c061a
  Stored in directory: /root/.cache/pip/wheels/bb/df/3f/81b36f41b66e6a9cd69224c70a737de2bb6b2f7feb3272c25e
  Created wheel for keras-self-attention: filename=keras_self_attention-0.41.0-cp36-none-any.whl size=17288 sha256=b891999bd64cb1c72dfd01897d603ee56f5e0886ec67731ea9ca533298551a5a
  Stored in directory: /root/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca696186635db694
Successfully built keras-multi-head keras-self-attention


In [4]:
import pandas as pd
import numpy as np
import json
import keras
import tensorflow as tf
import io
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from keras.preprocessing import sequence
from keras.layers import Conv1D, Conv2D
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import nltk
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from keras.layers import average
import tensorflow_hub as hub
from keras.layers import Average
from keras.layers import Concatenate
nltk.download('punkt')
from numpy import random
import math
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
from keras.layers import SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import gc
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.metrics import top_k_categorical_accuracy
from focal_loss import BinaryFocalLoss
from keras_multi_head import MultiHeadAttention
from keras_multi_head import MultiHead
#plt.switch_backend('agg')
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
!pip install keras_metrics
import keras_metrics as km

In [0]:

root_path='/content/drive/My Drive/offenseval/2020'

In [0]:
train_data=pd.read_csv(root_path+'/non_english_data/Danish/offenseval-da-training-v1.tsv',delimiter='\t',quoting=3)

In [0]:
test_data=pd.read_csv(root_path+'/non_english_data/Danish/offenseval-da-test-v1-nolabels.tsv',delimiter='\t',quoting=3)

In [0]:
test_data_labels=pd.read_csv(root_path+'/non_english_data/Danish/danish-goldlabels.csv',header=None,quoting=3)

In [0]:
# with open(root_path+'/non_english_data/Danish/offenseval-da-test-v1-nolabels.tsv','r') as f:
#   lines=f.readlines()

In [0]:
# len(lines)

In [0]:
print(test_data.shape)

(329, 2)


In [0]:
print(train_data.head())

     id                                              tweet subtask_a
0  3131  Jeg tror det vil være dejlig køligt, men jeg v...       NOT
1   711  Så kommer de nok til at investere i en ny cyke...       NOT
2  2500  Nu er det jo også de Ikea-aber der har lavet s...       OFF
3  2678  128 Varme emails, er vi enige om at det er sex...       NOT
4   784  Desværre tyder det på, at amerikanerne er helt...       NOT


## Cleaning

In [10]:
!pip install tqdm
from tqdm import tqdm

In [11]:
def remove_pattern(input_txt, pattern,with_space=False):
    r = re.findall(pattern, input_txt)
    if with_space==False:
      for i in r:
        input_txt = re.sub(i, '', input_txt)
    else:
      for i in r:
        input_txt = re.sub(i, ' ', input_txt)
    return input_txt 
!pip install emoji
import emoji
import pickle
import re
with open('/content/drive/My Drive/Sentimix/helper_data/contractions.pkl','rb')as f:
  contractions=pickle.load(f)


from collections import Counter
contractions=Counter(contractions)
with open('/content/drive/My Drive/Sentimix/helper_data/acronyms.pkl','rb')as f:
  acronyms=pickle.load(f)
acronyms=Counter(acronyms)
def acronym(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if acronyms[word]!=0:
        w_l.append(acronyms[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
# with open('/content/drive/My Drive/Sentimix/hinglish_to_english.pickle','rb')as f:
#   hing_to_eng=pickle.load(f)
# hing_to_eng=Counter(hing_to_eng)
def hindi_se_english(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    w_l=[]
    sent=str(df[column][i])
    for word in sent.split():
      if hing_to_eng[word]!=0:
        w_l.append(hing_to_eng[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
# with open('/content/drive/My Drive/Sentimix/Hinglish_utils/Hinglish_Profanity_dict.pkl', 'rb') as handle:
#     cuss_dict=pickle.load(handle)
# cuss_dict=Counter(cuss_dict)
def replace_cuss(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if cuss_dict[word]!=0:
        w_l.append('abuse')
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_contraction(df,column):
  s_l=[]
  for i in tqdm(range(df.shape[0])):
    sent=str(df[column][i]).lower()
    w_l=[]
    for word in sent.split():
      if contractions[word]!=0:
        w_l.append(contractions[word])
      else:
        w_l.append(word)
    s_l.append(' '.join(w_l))
  return s_l
def remove_pattern_rep(input_txt, pattern,rep_pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
      input_txt = re.sub(i, rep_pattern, input_txt)

    return input_txt 
def cleaning(data_f,cleaning_col,new_col):
  data_f.reset_index(drop=True,inplace=True)
  for i in tqdm(range(data_f.shape[0])):
    data_f[cleaning_col][i]=emoji.demojize(str(data_f[cleaning_col][i]))
  #data_f[cleaning_col]=replace_cuss(data_f,cleaning_col)
  data_f[new_col]=np.vectorize(remove_pattern)(data_f[cleaning_col],"_",with_space=True)
  data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],"-",with_space=True)
  data_f[new_col]=np.vectorize(remove_pattern)(data_f[new_col],":",with_space=True)
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "@[\w]*","<USR>")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[new_col], "http\S+","<URL>")
  data_f[new_col] = np.vectorize(remove_pattern_rep)(data_f[cleaning_col], "[0-9]+","<NUM>")
  #data_f[new_col]=hindi_se_english(data_f,cleaning_col)
  data_f[new_col]=remove_contraction(data_f,new_col)
  data_f[new_col]=acronym(data_f,new_col)
  data_f[new_col]=data_f[new_col].str.replace("[^a-zA-Z]<>", " ")
  data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "~",with_space=False)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], "!",with_space=True)
  #data_f[new_col] = np.vectorize(remove_pattern)(data_f[new_col], ".",with_space=True)
  #data_f[new_col] = data_f[new_col].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
  return data_f
import numpy as np
#a=cleaning(data,'text','clean_col')


     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=0d71875da7275c1a9753b6a1e5d19ad52dfea85056145f330f43deb3ebf43205
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [12]:
train_data=cleaning(train_data,'tweet','clean_col')

100%|██████████| 2961/2961 [00:00<00:00, 26768.63it/s]


In [13]:
test_data=cleaning(test_data,'tweet','clean_col')

  0%|          | 0/329 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 329/329 [00:00<00:00, 22636.21it/s]


In [14]:
import re, random
import spacy
nlp = spacy.load('en')

to_sample = False # if you're impatient switch this flag

def spacy_tokenize(text):
    return [token.text for token in nlp.tokenizer(text)]
    
def dameraulevenshtein(seq1, seq2):
    """Calculate the Damerau-Levenshtein distance between sequences.
    This method has not been modified from the original.
    Source: http://mwh.geek.nz/2009/04/26/python-damerau-levenshtein-distance/
    This distance is the number of additions, deletions, substitutions,
    and transpositions needed to transform the first sequence into the
    second. Although generally used with strings, any sequences of
    comparable objects will work.
    Transpositions are exchanges of *consecutive* characters; all other
    operations are self-explanatory.
    This implementation is O(N*M) time and O(M) space, for N and M the
    lengths of the two sequences.
    >>> dameraulevenshtein('ba', 'abc')
    2
    >>> dameraulevenshtein('fee', 'deed')
    2
    It works with arbitrary sequences too:
    >>> dameraulevenshtein('abcd', ['b', 'a', 'c', 'd', 'e'])
    2
    """
    # codesnippet:D0DE4716-B6E6-4161-9219-2903BF8F547F
    # Conceptually, this is based on a len(seq1) + 1 * len(seq2) + 1 matrix.
    # However, only the current and two previous rows are needed at once,
    # so we only store those.
    oneago = None
    thisrow = list(range(1, len(seq2) + 1)) + [0]
    for x in range(len(seq1)):
        # Python lists wrap around for negative indices, so put the
        # leftmost column at the *end* of the list. This matches with
        # the zero-indexed strings and saves extra calculation.
        twoago, oneago, thisrow = (oneago, thisrow, [0] * len(seq2) + [x + 1])
        for y in range(len(seq2)):
            delcost = oneago[y] + 1
            addcost = thisrow[y - 1] + 1
            subcost = oneago[y - 1] + (seq1[x] != seq2[y])
            thisrow[y] = min(delcost, addcost, subcost)
            # This block deals with transpositions
            if (x > 0 and y > 0 and seq1[x] == seq2[y - 1]
                    and seq1[x - 1] == seq2[y] and seq1[x] != seq2[y]):
                thisrow[y] = min(thisrow[y], twoago[y - 2] + 1)
    return thisrow[len(seq2) - 1]


class SymSpell:
    def __init__(self, max_edit_distance=3, verbose=0):
        self.max_edit_distance = max_edit_distance
        self.verbose = verbose
        # 0: top suggestion
        # 1: all suggestions of smallest edit distance
        # 2: all suggestions <= max_edit_distance (slower, no early termination)

        self.dictionary = {}
        self.longest_word_length = 0

    def get_deletes_list(self, w):
        """given a word, derive strings with up to max_edit_distance characters
           deleted"""

        deletes = []
        queue = [w]
        for d in range(self.max_edit_distance):
            temp_queue = []
            for word in queue:
                if len(word) > 1:
                    for c in range(len(word)):  # character index
                        word_minus_c = word[:c] + word[c + 1:]
                        if word_minus_c not in deletes:
                            deletes.append(word_minus_c)
                        if word_minus_c not in temp_queue:
                            temp_queue.append(word_minus_c)
            queue = temp_queue

        return deletes

    def create_dictionary_entry(self, w):
        '''add word and its derived deletions to dictionary'''
        # check if word is already in dictionary
        # dictionary entries are in the form: (list of suggested corrections,
        # frequency of word in corpus)
        new_real_word_added = False
        if w in self.dictionary:
            # increment count of word in corpus
            self.dictionary[w] = (self.dictionary[w][0], self.dictionary[w][1] + 1)
        else:
            self.dictionary[w] = ([], 1)
            self.longest_word_length = max(self.longest_word_length, len(w))

        if self.dictionary[w][1] == 1:
            # first appearance of word in corpus
            # n.b. word may already be in dictionary as a derived word
            # (deleting character from a real word)
            # but counter of frequency of word in corpus is not incremented
            # in those cases)
            new_real_word_added = True
            deletes = self.get_deletes_list(w)
            for item in deletes:
                if item in self.dictionary:
                    # add (correct) word to delete's suggested correction list
                    self.dictionary[item][0].append(w)
                else:
                    # note frequency of word in corpus is not incremented
                    self.dictionary[item] = ([w], 0)

        return new_real_word_added

    def create_dictionary_from_arr(self, arr, token_pattern=r'[a-z]+'):
        total_word_count = 0
        unique_word_count = 0

        for line in arr:
            # separate by words by non-alphabetical characters
            words = re.findall(token_pattern, line.lower())
            for word in words:
                total_word_count += 1
                if self.create_dictionary_entry(word):
                    unique_word_count += 1

        print("total words processed: %i" % total_word_count)
        print("total unique words in corpus: %i" % unique_word_count)
        print("total items in dictionary (corpus words and deletions): %i" % len(self.dictionary))
        print("  edit distance for deletions: %i" % self.max_edit_distance)
        print("  length of longest word in corpus: %i" % self.longest_word_length)
        return self.dictionary

    def create_dictionary(self, fname):
        total_word_count = 0
        unique_word_count = 0

        with open(fname) as file:
            for line in file:
                # separate by words by non-alphabetical characters
                words = re.findall('[a-z]+', line.lower())
                for word in words:
                    total_word_count += 1
                    if self.create_dictionary_entry(word):
                        unique_word_count += 1

        print("total words processed: %i" % total_word_count)
        print("total unique words in corpus: %i" % unique_word_count)
        print("total items in dictionary (corpus words and deletions): %i" % len(self.dictionary))
        print("  edit distance for deletions: %i" % self.max_edit_distance)
        print("  length of longest word in corpus: %i" % self.longest_word_length)
        return self.dictionary

    def get_suggestions(self, string, silent=False):
        """return list of suggested corrections for potentially incorrectly
           spelled word"""
        if (len(string) - self.longest_word_length) > self.max_edit_distance:
            if not silent:
                print("no items in dictionary within maximum edit distance")
            return []

        suggest_dict = {}
        min_suggest_len = float('inf')

        queue = [string]
        q_dictionary = {}  # items other than string that we've checked

        while len(queue) > 0:
            q_item = queue[0]  # pop
            queue = queue[1:]

            # early exit
            if ((self.verbose < 2) and (len(suggest_dict) > 0) and
                    ((len(string) - len(q_item)) > min_suggest_len)):
                break

            # process queue item
            if (q_item in self.dictionary) and (q_item not in suggest_dict):
                if self.dictionary[q_item][1] > 0:
                    # word is in dictionary, and is a word from the corpus, and
                    # not already in suggestion list so add to suggestion
                    # dictionary, indexed by the word with value (frequency in
                    # corpus, edit distance)
                    # note q_items that are not the input string are shorter
                    # than input string since only deletes are added (unless
                    # manual dictionary corrections are added)
                    assert len(string) >= len(q_item)
                    suggest_dict[q_item] = (self.dictionary[q_item][1],
                                            len(string) - len(q_item))
                    # early exit
                    if (self.verbose < 2) and (len(string) == len(q_item)):
                        break
                    elif (len(string) - len(q_item)) < min_suggest_len:
                        min_suggest_len = len(string) - len(q_item)

                # the suggested corrections for q_item as stored in
                # dictionary (whether or not q_item itself is a valid word
                # or merely a delete) can be valid corrections
                for sc_item in self.dictionary[q_item][0]:
                    if sc_item not in suggest_dict:

                        # compute edit distance
                        # suggested items should always be longer
                        # (unless manual corrections are added)
                        assert len(sc_item) > len(q_item)

                        # q_items that are not input should be shorter
                        # than original string
                        # (unless manual corrections added)
                        assert len(q_item) <= len(string)

                        if len(q_item) == len(string):
                            assert q_item == string
                            item_dist = len(sc_item) - len(q_item)

                        # item in suggestions list should not be the same as
                        # the string itself
                        assert sc_item != string

                        # calculate edit distance using, for example,
                        # Damerau-Levenshtein distance
                        item_dist = dameraulevenshtein(sc_item, string)

                        # do not add words with greater edit distance if
                        # verbose setting not on
                        if (self.verbose < 2) and (item_dist > min_suggest_len):
                            pass
                        elif item_dist <= self.max_edit_distance:
                            assert sc_item in self.dictionary  # should already be in dictionary if in suggestion list
                            suggest_dict[sc_item] = (self.dictionary[sc_item][1], item_dist)
                            if item_dist < min_suggest_len:
                                min_suggest_len = item_dist

                        # depending on order words are processed, some words
                        # with different edit distances may be entered into
                        # suggestions; trim suggestion dictionary if verbose
                        # setting not on
                        if self.verbose < 2:
                            suggest_dict = {k: v for k, v in suggest_dict.items() if v[1] <= min_suggest_len}

            # now generate deletes (e.g. a substring of string or of a delete)
            # from the queue item
            # as additional items to check -- add to end of queue
            assert len(string) >= len(q_item)

            # do not add words with greater edit distance if verbose setting
            # is not on
            if (self.verbose < 2) and ((len(string) - len(q_item)) > min_suggest_len):
                pass
            elif (len(string) - len(q_item)) < self.max_edit_distance and len(q_item) > 1:
                for c in range(len(q_item)):  # character index
                    word_minus_c = q_item[:c] + q_item[c + 1:]
                    if word_minus_c not in q_dictionary:
                        queue.append(word_minus_c)
                        q_dictionary[word_minus_c] = None  # arbitrary value, just to identify we checked this

        # queue is now empty: convert suggestions in dictionary to
        # list for output
        if not silent and self.verbose != 0:
            print("number of possible corrections: %i" % len(suggest_dict))
            print("  edit distance for deletions: %i" % self.max_edit_distance)

        # output option 1
        # sort results by ascending order of edit distance and descending
        # order of frequency
        #     and return list of suggested word corrections only:
        # return sorted(suggest_dict, key = lambda x:
        #               (suggest_dict[x][1], -suggest_dict[x][0]))

        # output option 2
        # return list of suggestions with (correction,
        #                                  (frequency in corpus, edit distance)):
        as_list = suggest_dict.items()
        # outlist = sorted(as_list, key=lambda (term, (freq, dist)): (dist, -freq))
        outlist = sorted(as_list, key=lambda x: (x[1][1], -x[1][0]))

        if self.verbose == 0:
            return outlist[0]
        else:
            return outlist

        '''
        Option 1:
        ['file', 'five', 'fire', 'fine', ...]
        Option 2:
        [('file', (5, 0)),
         ('five', (67, 1)),
         ('fire', (54, 1)),
         ('fine', (17, 1))...]  
        '''

    def best_word(self, s, silent=False):
        try:
            return self.get_suggestions(s, silent)[0]
        except:
            return None

def spell_corrector(word_list, words_d) -> str:
    result_list = []
    for word in word_list:
        if word not in words_d:
            suggestion = ss.best_word(word, silent=True)
            if suggestion is not None:
                result_list.append(suggestion)
        else:
            result_list.append(word)
            
    return " ".join(result_list)

if __name__ == '__main__':
    # build symspell tree 
    ss = SymSpell(max_edit_distance=2)
    
    # fetch list of bad words
    with open('/content/drive/My Drive/offenseval/2020/cleaner_data/bad-words.csv') as bf:
        bad_words = bf.readlines()
    bad_words = [word.strip() for word in bad_words]    
    
    # fetch english words dictionary
    with open('/content/drive/My Drive/offenseval/2020/cleaner_data/english_words_479k.txt') as f:
        words = f.readlines()
    eng_words = [word.strip() for word in words]
    
    # Print some examples
    print(eng_words[:5])
    print(bad_words[:5])

    print('Total english words: {}'.format(len(eng_words)))
    print('Total bad words: {}'.format(len(bad_words)))
    
    print('create symspell dict...')
    
    if to_sample:
        # sampling from list for kernel runtime
        sample_idxs = random.sample(range(len(eng_words)), 100)
        eng_words = [eng_words[i] for i in sorted(sample_idxs)] + \
            'to infinity and beyond'.split() # make sure our sample misspell is in there
    
    all_words_list = list(set(bad_words + eng_words))
    silence = ss.create_dictionary_from_arr(all_words_list, token_pattern=r'.+')
    
    # create a dictionary of rightly spelled words for lookup
    words_dict = {k: 0 for k in all_words_list}
    
    sample_text = 'to infifity and byond'
    


['2', '1080', '&c', '10-point', '10th']
['jigaboo', 'mound of venus', 'asslover', 's&m', 'queaf']
Total english words: 466544
Total bad words: 1617
create symspell dict...
total words processed: 467594
total unique words in corpus: 467394
total items in dictionary (corpus words and deletions): 20250415
  edit distance for deletions: 2
  length of longest word in corpus: 45


In [15]:
test_data['tokens'] = test_data['clean_col'].apply(spacy_tokenize)
    
print('run spell checker...')
print()
#print('original text: ' + sample_text)
print()
test_data['clean'] = test_data.apply(lambda row:spell_corrector(row['tokens'],words_dict),axis=1)
#print('corrected text: ' + correct_text)

print('Done.')  

run spell checker...


Done.


In [16]:
train_data['tokens'] = train_data['clean_col'].apply(spacy_tokenize)
    
print('run spell checker...')
print()
#print('original text: ' + sample_text)
print()
train_data['clean'] = train_data.apply(lambda row:spell_corrector(row['tokens'],words_dict),axis=1)
#print('corrected text: ' + correct_text)

print('Done.')  

run spell checker...


Done.


## Tokenization and Split

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
from sklearn.model_selection import StratifiedKFold

In [0]:
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)

In [0]:
df_train,df_dev=train_test_split(train_data,test_size=0.1,random_state=2020)

In [0]:
y_train=(df_train['subtask_a']=="OFF").astype(int)
y_dev=(df_dev['subtask_a']=="OFF").astype(int)

In [22]:
test_data_labels.head()

,0,1
0,1382,NOT
1,1384,NOT
2,547,NOT
3,1269,NOT
4,1695,OFF


In [0]:
y_test=(test_data_labels[1]=="OFF").astype(int)

In [24]:
print(train_data.shape)

(2961, 6)


In [0]:
max_words =3000
max_len = 45
tok = Tokenizer(max_words)
tok.fit_on_texts(train_data['clean'].astype(str))

In [0]:
sequences_train = tok.texts_to_sequences(df_train['clean'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len,padding='post',truncating='post')

In [0]:
sequences_dev = tok.texts_to_sequences(df_dev['clean'].astype(str))
vocab_size = len(tok.word_index) + 1
sequences_matrix_dev = sequence.pad_sequences(sequences_dev,maxlen=max_len,padding='post',truncating='post')

In [0]:
sequences_test = tok.texts_to_sequences(test_data['clean'].astype(str))
# vocab_size = len(tok.word_index) + 1
sequences_matrix_test = sequence.pad_sequences(sequences_test,maxlen=max_len,padding='post',truncating='post')

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

## Model

In [0]:
word_index=tok.word_index

In [113]:
print(word_index)

{'c': 1, 'er': 2, 'det': 3, 'at': 4, 'i': 5, 'original': 6, 'gangster': 7, 'en': 8, 'jeg': 9, 'p': 10, 'kike': 11, 'de': 12, 'there': 13, 'until': 14, 'for': 15, 's': 16, 'med': 17, 'hear': 18, 'den': 19, 'aface': 20, 'som': 21, 'kan': 22, 'num': 23, 'du': 24, 'et': 25, 'vi': 26, 'men': 27, 'man': 28, 'e': 29, 'om': 30, 'var': 31, 'skal': 32, 'vare': 33, 'the': 34, 'a': 35, 'h': 36, 'god': 37, 'r': 38, 'how': 39, 'about': 40, 'now': 41, 'vis': 42, 'heller': 43, 'vor': 44, 'bare': 45, 'togs': 46, 'her': 47, 'fra': 48, 'vad': 49, 'jo': 50, 'soget': 51, 'w': 52, 'nr': 53, 'lige': 54, 'user': 55, 'dansker': 56, 'veld': 57, 'danmark': 58, 'ud': 59, 'vil': 60, 'alle': 61, 'fr': 62, 'gr': 63, 'reget': 64, 'new': 65, 'signature': 66, 'mig': 67, 'honey': 68, 'gunne': 69, 'have': 70, 'ladt': 71, 'over': 72, 'selv': 73, 'blijver': 74, 'xeres': 75, 'm': 76, 'svensen': 77, 'dig': 78, 'nook': 79, 'them': 80, 'ville': 81, 'minute': 82, 'fu': 83, 'se': 84, 'operator': 85, 'kummer': 86, 'hell': 87, 'da

In [114]:
!pip install keras-self-attention

In [0]:
from keras.layers import Concatenate
from sklearn.metrics import precision_recall_fscore_support

In [0]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras_self_attention import SeqSelfAttention
from keras.layers import CuDNNGRU,CuDNNLSTM,GlobalMaxPooling1D,GlobalAveragePooling1D
from sklearn.utils import class_weight
class Attention(Layer):
    def __init__(self,step_dim=20,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, Dropout

#max_len=

def BidLstm(maxlen, max_features, embed_size):
    inp1 = Input(shape=(maxlen, ))
    #inp2=Input(shape=(1,))
    x1=Embedding(max_words+1,embed_size)(inp1)
    #x1 = Embedding(max_words + 1,embed_size,weights=[embedding_matrix_1],
    #              trainable=True)(inp1)
    # x2 = Embedding(len(tok.word_index) + 1,embed_size_2,weights=[embedding_matrix_2],
    #                trainable=True)(inp1)
    # x3 = Embedding(len(tok.word_index) + 1,embed_size_3,weights=[embedding_matrix_3],
    #                trainable=True)(inp1)
    # x1 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x1)
    # x2 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x2)
    # x3 = Bidirectional(LSTM(200, return_sequences=True, dropout=0.4,
    #                        recurrent_dropout=0.4))(x3)   
    #x1 = Attention(maxlen)(x1)
    # x2 = Attention(maxlen)(x2)
    # x3 = Attention(maxlen)(x3)
    # x=  Concatenate()([x1,x2,x3])
    # x1=MultiHead(layer=keras.layers.Bidirectional(keras.layers.CuDNNLSTM(units=200), name='LSTM'),
    # layer_num=5,
    # reg_index=[1, 4],
    # reg_slice=(slice(None, None), slice(32, 48)),
    # reg_factor=0.1,
    # name='Multi-Head-Attention')(x1)
    x1 = CuDNNLSTM(256, return_sequences=True)(x1)   
    x1 = SeqSelfAttention(kernel_regularizer=keras.regularizers.l2(1e-4),
                    bias_regularizer=keras.regularizers.l1(1e-4),
                    attention_regularizer_weight=1e-4,
                    name='Attention')(x1) 
    
    x2=  GlobalMaxPooling1D()(x1)
    x3= GlobalAveragePooling1D()(x1)
    x=  Concatenate()([x2,x3])
    x = Dropout(0.1)(x)
    #x = Attention(maxlen)(x)
    # layer = Dense(600,name='FC1')(x)
    # layer = Dense(300,activation='relu')(layer)
    layer = Dense(128,activation='relu')(x)
 #   layer = BatchNormalization(name = 'BN1')(layer)
    #layer = Activation('relu')(layer)
    #layer = Dropout(0.4)(layer)
    layer = Dense(64,name='FC2')(layer)
#    layer = BatchNormalization(name = 'BN2')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.4)(layer)
   # layer=  Concatenate()([layer,inp2])
    # layer=Dense(256,activation='relu')(layer)
    # layer=Dense(128,activation='relu')(layer)
    layer = Dense(1,name='out_layer',activation='sigmoid')(layer)

    model = Model(inputs=[inp1],outputs=layer)

    return model
model_bi=BidLstm(max_len,max_features=max_words,embed_size=300)

In [145]:
model_bi.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc',km.f1_score()])

tracking <tf.Variable 'Variable_36:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_37:0' shape=() dtype=int32> fp
tracking <tf.Variable 'Variable_38:0' shape=() dtype=int32> tp
tracking <tf.Variable 'Variable_39:0' shape=() dtype=int32> fn


In [146]:
model_bi.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 45)           0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 45, 300)      900300      input_9[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm_9 (CuDNNLSTM)        (None, 45, 256)      571392      embedding_9[0][0]                
__________________________________________________________________________________________________
Attention (SeqSelfAttention)    (None, 45, 256)      16449       cu_dnnlstm_9[0][0]               
____________________________________________________________________________________________

In [0]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights=dict(enumerate(class_weights))

In [148]:
print(class_weights)

{0: 0.5733964700817907, 1: 3.906158357771261}


In [0]:
class_weights[1]=3.5

In [0]:
cp_filepath='/content/drive/My Drive/offenseval/'+'checkpoints/lstm_model_2020a_danish.h5'
cp_check_point=keras.callbacks.ModelCheckpoint(cp_filepath, monitor='val_f1_score', verbose=0, save_best_only=True, save_weights_only=False, mode='max', period=1)
es = EarlyStopping(monitor='val_f1_score', mode='max', min_delta=0,patience=2,restore_best_weights=True)
reduce_lr=keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)

In [151]:
model_bi.fit([sequences_matrix_train],y_train,validation_data=([sequences_matrix_dev],y_dev),epochs=2,batch_size=32,class_weight=class_weights,callbacks=[es,cp_check_point])

Train on 2664 samples, validate on 297 samples
Epoch 1/2
2664/2664 [==============================] - 3s 1ms/step - loss: 0.6692 - acc: 0.8074 - f1_score: 0.1107 - val_loss: 0.6536 - val_acc: 0.7374 - val_f1_score: 0.2747
Epoch 2/2
2664/2664 [==============================] - 1s 540us/step - loss: 0.5549 - acc: 0.7920 - f1_score: 0.3653 - val_loss: 0.4682 - val_acc: 0.8620 - val_f1_score: 0.5548


In [0]:
y_preds_dl=model_bi.predict(sequences_matrix_dev)

In [153]:
from sklearn.metrics import classification_report

y_pred = model_bi.predict(sequences_matrix_dev, batch_size=30, verbose=1)

print(classification_report(y_dev, y_pred.round(),digits=4))

297/297 [==============================] - 0s 128us/step
              precision    recall  f1-score   support

           0     0.9176    0.9213    0.9194       254
           1     0.5238    0.5116    0.5176        43

    accuracy                         0.8620       297
   macro avg     0.7207    0.7164    0.7185       297
weighted avg     0.8606    0.8620    0.8613       297



In [0]:
preds=model_bi.predict([sequences_matrix_test])

In [95]:
print(classification_report(y_test, preds.round(),digits=4))

              precision    recall  f1-score   support

           0     0.9234    0.8785    0.9004       288
           1     0.3636    0.4878    0.4167        41

    accuracy                         0.8298       329
   macro avg     0.6435    0.6831    0.6585       329
weighted avg     0.8536    0.8298    0.8401       329



In [0]:
a=[]
for i in range(len(preds)):
  if preds[i].round()==1:
    a.append("OFF")
  else:
    a.append("NOT")

In [0]:
df=pd.DataFrame({"ids":test_data['id'],"preds":a},index=None)
df.to_csv(root_path+'/non_english_data/Danish/submission.csv',index=False)